<a href="https://colab.research.google.com/github/lzf834/VS-Code-Projects/blob/main/GamersUnite.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [24]:
%%shell
# Ubuntu no longer distributes chromium-browser outside of snap
#
# Proposed solution: https://askubuntu.com/questions/1204571/how-to-install-chromium-without-snap

# Add debian buster
cat > /etc/apt/sources.list.d/debian.list <<'EOF'
deb [arch=amd64 signed-by=/usr/share/keyrings/debian-buster.gpg] http://deb.debian.org/debian buster main
deb [arch=amd64 signed-by=/usr/share/keyrings/debian-buster-updates.gpg] http://deb.debian.org/debian buster-updates main
deb [arch=amd64 signed-by=/usr/share/keyrings/debian-security-buster.gpg] http://deb.debian.org/debian-security buster/updates main
EOF

# Add keys
apt-key adv --keyserver keyserver.ubuntu.com --recv-keys DCC9EFBF77E11517
apt-key adv --keyserver keyserver.ubuntu.com --recv-keys 648ACFD622F3D138
apt-key adv --keyserver keyserver.ubuntu.com --recv-keys 112695A0E562B32A

apt-key export 77E11517 | gpg --dearmour -o /usr/share/keyrings/debian-buster.gpg
apt-key export 22F3D138 | gpg --dearmour -o /usr/share/keyrings/debian-buster-updates.gpg
apt-key export E562B32A | gpg --dearmour -o /usr/share/keyrings/debian-security-buster.gpg

# Prefer debian repo for chromium* packages only
# Note the double-blank lines between entries
cat > /etc/apt/preferences.d/chromium.pref << 'EOF'
Package: *
Pin: release a=eoan
Pin-Priority: 500


Package: *
Pin: origin "deb.debian.org"
Pin-Priority: 300


Package: chromium*
Pin: origin "deb.debian.org"
Pin-Priority: 700
EOF

# Install chromium and chromium-driver
apt-get update
apt-get install chromium chromium-driver

# Install selenium
pip install selenium

Executing: /tmp/apt-key-gpghome.9ELwHFfE0B/gpg.1.sh --keyserver keyserver.ubuntu.com --recv-keys DCC9EFBF77E11517
gpg: key DCC9EFBF77E11517: public key "Debian Stable Release Key (10/buster) <debian-release@lists.debian.org>" imported
gpg: Total number processed: 1
gpg:               imported: 1
Executing: /tmp/apt-key-gpghome.7sfuaIEJIT/gpg.1.sh --keyserver keyserver.ubuntu.com --recv-keys 648ACFD622F3D138
gpg: key DC30D7C23CBBABEE: public key "Debian Archive Automatic Signing Key (10/buster) <ftpmaster@debian.org>" imported
gpg: Total number processed: 1
gpg:               imported: 1
Executing: /tmp/apt-key-gpghome.sTex2LIqFn/gpg.1.sh --keyserver keyserver.ubuntu.com --recv-keys 112695A0E562B32A
gpg: key 4DFAB270CAA96DFA: public key "Debian Security Archive Automatic Signing Key (10/buster) <ftpmaster@debian.org>" imported
gpg: Total number processed: 1
gpg:               imported: 1
Get:1 http://deb.debian.org/debian buster InRelease [122 kB]
Get:2 http://deb.debian.org/debian bust

In [25]:
from selenium import webdriver
from selenium.webdriver.common.by import By
import time
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--headless')
chrome_options.add_argument('--no-sandbox')
chrome_options.headless = True
wd = webdriver.Chrome('chromedriver',options=chrome_options)
wd.get("https://shuttleonline.ktmb.com.my/Home/Shuttle")

<ipython-input-25-7683da42514f>:7: DeprecationWarning: headless property is deprecated, instead use add_argument('--headless') or add_argument('--headless=new')
  chrome_options.headless = True


SessionNotCreatedException: ignored

In [ ]:
time.sleep(5)


In [19]:
loc_obtained = False

while not loc_obtained:
  ### Get user locations
  desired_departure_location = input("Please input WOODLANDS CIQ, JB SENTRAL or BOTH")
  desired_departure_location = desired_departure_location.upper()

  ### Location Validation
  two_way = False
  click_swap = False
  if desired_departure_location == "BOTH":
    two_way = True
    click_swap = True
    print("2 ways selected: departing from Woodlands")

  elif desired_departure_location == "WOODLANDS CIQ":
    click_swap = True
    print("1 way selected: departing from Woodlands")

  elif desired_departure_location != "JB SENTRAL":
    print("Please enter a valid location input")
    continue

  else:
    print("1 way selected: departing from JB Sentral")
  loc_obtained = True



Please input WOODLANDS CIQ, JB SENTRAL or BOTHboth
2 ways selected: departing from Woodlands


In [21]:
from string import digits
from datetime import datetime

dates_obtained = False
allowed = set(digits).union('/')
today = datetime.now()
print(today.strftime("%d/%m/%y"))

while not dates_obtained:
  print("Please input date in dd/mm/yy format")

  ### Get user dates
  desired_depart_date = input("When do you wanna leave?")

  if two_way:
    desired_arrival_date = input("When do u wanna come back?")

  ### Dates validation
  if not all(c in allowed for c in desired_depart_date):
    print("please enter desired depart date in a proper format")
    continue
  elif two_way and not all(c in allowed for c in desired_arrival_date):
    print("please enter desired arrival date in a proper format")
    continue
  
  ## check date is logical -> future from now
  depart_date_obj = datetime.strptime(desired_depart_date, "%d/%m/%y")
  if depart_date_obj.date() < today.date():
    print("please input a date that is today or in the future")
    continue

  if two_way:
    arrival_date_obj = datetime.strptime(desired_arrival_date, "%d/%m/%y")
    if arrival_date_obj.date() < depart_date_obj.date():
      print("arrival date must be same or further than departure date")
      continue

  dates_obtained = True

23/05/23
Please input date in dd/mm/yy format
When do you wanna leave?25/5/23
When do u wanna come back?27/5/23
2023-05-23 02:48:44.611260 2023-05-25 00:00:00


In [ ]:
### Current Departure before click
departure_location_list = wd.find_elements(By.XPATH, "/html/body/div[2]/div/form/div/div[1]/input")
for input in departure_location_list:
  departure_location = input.get_attribute("value")
  

In [22]:
if click_swap:
  print("exchange button will be clicked now!....")
  exhangeButton = wd.find_elements(By.XPATH, "/html/body/div[2]/div/form/div/div[1]/i")
  for exBtn in exhangeButton:
    exBtn.click()
  

exchange button will be clicked now!....


NameError: ignored

In [ ]:
departure_location_list = wd.find_elements(By.XPATH, "/html/body/div[2]/div/form/div/div[1]/input")
for input in departure_location_list:
  departure_location = input.get_attribute("value")
if departure_location == desired_departure_location or desired_departure_location == "BOTH":
  print("Desired departure location is now selected successfully....")
else:
  print("OOPS something went wrong.... restart the program....")

Desired departure location is now selected successfully....


In [ ]:
desired_depart_date = desired_depart_date.split("/")
continueBool = all(ele.isdigit() for ele in desired_depart_date)
print(desired_depart_date)
if continueBool == False:
  print("please restart this program and enter a valid date next time....")
# for date in desired_depart_date:
#   numericbool = date.isnumeric()
#   if !numericbool:
#     print("please restart the program and enter a valid date....")
#     continueBool = False
#   else:
#     continueBool = True

['29', '5', '2023']


In [ ]:
if continueBool:
  departDateInput = wd.find_elements(By.XPATH, "/html/body/div[2]/div/form/div/div[5]/div/div[1]/input")
  for departDateInputDiv in departDateInput:
    departDateInputDiv.click()

In [ ]:
string_var = "//div[contains(@class, 'lightpick__day') and text() = '%s']" % desired_depart_date[0]
departDateList = wd.find_elements(By.XPATH, string_var)
print(string_var) 
month_select_div = wd.find_elements(By.XPATH, "/html/body/section/div/div[1]/section/header/div[1]/select[1]")
for month in month_select_div:
  print(month.get_attribute("value"))
  current_depart_month_value = month.get_attribute("value")

for i in range(int(desired_depart_date[1]) - int(current_depart_month_value) - 1):
  button_arrow_month = wd.find_elements(By.XPATH, "/html/body/div[2]/div/form/div/div[5]/div/div[1]/input")
  for button in button_arrow_month:
    button.click()


month_select_div = wd.find_elements(By.XPATH, "/html/body/section/div/div[1]/section/header/div[1]/select[1]")
for month in month_select_div:
  print(month.get_attribute("value"))
  current_depart_month_value = month.get_attribute("value")



for div in departDateList:
  print("LOL")
  div.click()

//div[contains(@class, 'lightpick__day') and text() = '29']
4
4
LOL
